In [3]:
import numpy as np 
import time 
import pymanopt
from pymanopt import manifolds, optimizers, tools, core
from pymanopt.core.problem  import Problem
from typing import Sequence
from scipy.spatial.transform import Rotation as R

import networkx as nx
from useful_function_joint import *
from scipy import optimize
from scipy.optimize import Bounds

deg2rad = np.pi/180.
rad2deg = 180./np.pi

%load_ext autoreload
%autoreload 2

In [4]:
def load_kuka_KR_10_R1100(scale_shift=10,joints_ref=np.array([0,-90,90,0,0,0])*np.pi/180):

    e1 = np.array((1,0,0))*scale_shift
    e2 = np.array((0,1,0))*scale_shift
    e3 = np.array((0,0,1))*scale_shift
    e0 = np.array([0,0,0])

    t1 = np.array((25,0,0))
    t2 = np.array((560,0,0))
    t3 = np.array((515,0,25))
    t4 = np.array((0,0,0))
    t5 = np.array((90,0,0))

    R1 = Rz(joints_ref[0])
    R2 = Ry(joints_ref[1])
    R3 = Ry(joints_ref[2])
    R4 = Rx(joints_ref[3])
    R5 = Ry(joints_ref[4])
    R6 = Rx(joints_ref[5])

    T1 = T(t1)
    T2 = T(t2)
    T3 = T(t3)
    T4 = T(t4)
    T5 = T(t5)

    axes = ['z','y','y','x','y','x']

    translation = [T(e0),T1,T2,T3,T4,T5]
    rotation = [R1,R2,R3,R4,R5,R6]

    step_vect = {'z': [e0,e3],
                'y': [e2,-e2*0],
                'x': [e0,-e1]}
    
    return axes, translation, rotation, step_vect, e0, e1, e2, e3

In [5]:
def sol_by_edge(Y_star,new_edge_order,n_anchor,e):

    for i in range(len(new_edge_order)):
        if (e[0] == new_edge_order[i][0]) and (e[1] == new_edge_order[i][1]):
            if i < n_anchor:
                return Y_star[0].transpose()[i,:]
            else:
                return Y_star[1].transpose()[i-n_anchor,:]


def sol_to_config_kuka_KR_10_R1100(G,Y_star,new_edge_order,n_anchor,scale_shift=10):

    e1 = np.array((1,0,0))*scale_shift
    e2 = np.array((0,1,0))*scale_shift
    e3 = np.array((0,0,1))*scale_shift
    e0 = np.array([0,0,0])

    t1 = np.array((25,0,0))
    t2 = np.array((560,0,0))
    t3 = np.array((515,0,25))
    t4 = np.array((0,0,0))
    t5 = np.array((90,0,0))

    R1 = Rz
    R2 = Ry
    R3 = Ry
    R4 = Rx
    R5 = Ry
    R6 = Rx

    T1 = T(t1)
    T2 = T(t2)
    T3 = T(t3)
    T4 = T(t4)
    T5 = T(t5)

    axes = ['z','y','y','x','y','x']

    translation = [T(e0),T1,T2,T3,T4,T5]
    rotation = [R1,R2,R3,R4,R5,R6]

    step_vect = {'z': [e0,e3],
                'y': [e2,-e2*0],
                'x': [e0,-e1]}
    
    dir_ref = nx.get_edge_attributes(G,'dir_ref')
    
    v3 = norm(np.array([ 5.40000000e+02, -9.42477796e-04,  5.85000000e+02])-np.array([ 2.50000000e+01, -4.36332313e-05,  5.60000000e+02]))
    print('v3',  np.array([ 5.40000000e+02, -9.42477796e-04,  5.85000000e+02])-np.array([ 2.50000000e+01, -4.36332313e-05,  5.60000000e+02]), norm(np.array([ 5.40000000e+02, -9.42477796e-04,  5.85000000e+02])-np.array([ 2.50000000e+01, -4.36332313e-05,  5.60000000e+02])))
    reference_edges = [('a2_minus', 'a2_plus'), ('a2_minus', 'a3_minus'),('a3_minus', 'a4_origin'), ('a5_minus', 'a5_plus'),('a6_minus', 'a6_origin')]
    reference_direction = [norm(e2),norm(e1),norm(e1),norm(e2),norm(e1)]
    
    RR = Ry(-np.arctan2(v3[2],v3[0]))[:3,:3]
    directions_sol = [sol_by_edge(Y_star,new_edge_order,n_anchor,e) for e in reference_edges]
    print(directions_sol)
    ww = directions_sol[0]
    joints = [np.arctan2(ww[0],ww[1])]
    print('e1',rotation[0](joints[-1])[:3,:3].transpose()@ww)
    print('a1',joints[0]*180/np.pi,ww)
    y_list =  [1,4]
    for i in range(1,5):
        transform = []
        for j in reversed(range(i)):
            transform.append(rotation[j](-joints[j])[:3,:3])
        if len(transform)>1:
            transform = np.array(transform)
            transformation = np.linalg.multi_dot(transform)
        else:
            transformation = transform[0]
        #joints.append(np.arccos((np.linalg.inv(transformation)@directions_sol[i])@reference_direction[i]))
        ww = ((transformation)@directions_sol[i])
        print('ww:',ww,directions_sol[i])
        if i in y_list:
            print(ww)
            joints.append(-np.arctan2(ww[2],ww[0]))
        elif i == 2:
            ww = RR.transpose()@ww
            print(ww)
            joints.append(-np.arctan2(ww[2],ww[0]))
        else:
            joints.append(np.arctan2(ww[2],ww[1]))

        print(i,rotation[i](joints[-1])[:3,:3].transpose()@ww,reference_direction[i])
        print('----------------------------')
    
    return joints

In [24]:
joint = sol_to_config_kuka_KR_10_R1100(G,Y_D3,new_edge_order,n_anchor,scale_shift)
joint.append(0)
#joint[1] = -joint[1]
#joint[-2] = np.pi-joint[-2]
print([180/np.pi*j for j in joint])
print(pose_robot_(joint,'camera'))
print(position,direction)
print(np.linalg.norm(pose_robot_(joint,'camera')[:3,3] - position), np.linalg.norm(direction - pose_robot_(joint,'camera')[:3,0]))

v3 [ 5.15000000e+02 -8.98844565e-04  2.50000000e+01] [ 9.98823833e-01 -1.74327645e-06  4.84865939e-02]


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [23]:
hh = [('a2_minus', 'a2_plus'),
 ('a2_minus', 'a3_minus'),
 ('a2_minus', 'a3_plus'),
 ('a2_plus', 'a3_minus'),
 ('a2_plus', 'a3_plus'),
 ('a3_minus', 'a3_plus')]

y = []
for i,e in enumerate(new_edge_order):
    if e in hh:
        y.append(Y_D3.transpose()[i,:])

print(np.array(y).shape)
y = np.array(y)
print(np.linalg.svd(y.transpose()@y)[1])

(6, 3)
[3.34228150e+00 2.65771698e+00 1.52396237e-06]


In [6]:
def recover_config(scale_shift,X):

    joints_ref = np.array([0,0,0,0,0,0])
    axes, translation, rotation, step_vect, e0, e1, e2, e3 = load_kuka_KR_10_R1100(scale_shift,joints_ref)
    pose_ref_ax = pose_axes(joints_ref)
    #pose_ref_ax = [p[:3,3] for p in pose_ref_ax]

    rotation = [Rz, Ry, Ry, Rx, Ry, Rx]
    d_rotation = [dRz, dRy1, dRy1, dRx1,dRy1, dRx1]


    def cost(v_,X_goal):
        
        v = [v_l for v_l in v_]
        v.append(0)
        v = np.array(v)
        #v = np.array([90,-90,90,0,0,0])*deg2rad
        pose_ax_v = pose_axes(v)
        
        X = []

        for j in range(6):
            ax = axes[j]
            v1,v2 = step_vect[ax] 
            #print(pose)
            X.append(pose_ax_v[j][:3,3] + pose_ax_v[j][:3,:3]@v2)
            X.append(pose_ax_v[j][:3,3] + pose_ax_v[j][:3,:3]@v1)
        
        #print(np.array(X))
        X = np.array(X) - X_goal[2:,:]
        
        return np.linalg.norm(X)**2
    
    min_absolute_bound = np.array([-180.0, -189.75,15,-183.5,-119.5])*deg2rad
    max_absolute_bound = np.array([180.0, -25,155.5,183.5,119.5])*deg2rad
    
    bounds = Bounds(min_absolute_bound, max_absolute_bound)

    cost_goal = lambda  x: cost(x,X)
    res = optimize.minimize(
            cost_goal,
            # cost,
            x0 = np.array([90,-90,90,0,0])*deg2rad,
            jac='3-point',
            # jac=grad,
            bounds = bounds,
            method='Nelder-Mead',
            options={"xatol": 1e-7, "fatol": 1e-7, 'disp': True,'maxiter': 1000},
        )
    
    print(res.fun)
    return res.x

    

In [22]:

recover_config(scale_shift,X)


Optimization terminated successfully.
         Current function value: 5150189.682170
         Iterations: 710
         Function evaluations: 1118
5150189.682170348


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/scipy/optimize/_minimize.py:550: RuntimeWarning: Method Nelder-Mead does not use gradient information (jac).
  warn('Method %s does not use gradient information (jac).' % method,


array([ 3.14159265, -1.47486428,  0.26179939,  0.97554455, -0.37638745])

In [21]:
np.linalg.svd(X[4:8,:],full_matrices=False)

SVDResult(U=array([[ 0.00975122,  0.0337248 , -0.9192485 ],
       [ 0.56935465, -0.57690723, -0.24252198],
       [ 0.22968742,  0.79501185, -0.19324257],
       [ 0.78929328,  0.18438233,  0.24253368]]), S=array([1246.53150996,  641.40711891,    3.32403318]), Vh=array([[-0.87148908, -0.48704347,  0.0574059 ],
       [ 0.49040616, -0.86478344,  0.10794165],
       [-0.0029286 ,  0.12222218,  0.99249844]]))

In [59]:
nx.get_node_attributes(G,'pos_ref')

{'x': array([100,   0,   0]),
 'y': array([  0, 100,   0]),
 'a1_plus': array([  0.,   0., 100.]),
 'a1_origin': array([0., 0., 0.]),
 'a2_minus': array([ 1.5308085e-15, -2.5000000e+01,  0.0000000e+00]),
 'a2_plus': array([100., -25.,   0.]),
 'a3_minus': array([ 1.5308085e-15, -2.5000000e+01,  5.6000000e+02]),
 'a3_plus': array([100., -25., 560.]),
 'a4_minus': array([ 2.69422296e-14, -4.40000000e+02,  5.85000000e+02]),
 'a4_origin': array([ 3.30654636e-14, -5.40000000e+02,  5.85000000e+02]),
 'a5_minus': array([ 3.30654636e-14, -5.40000000e+02,  5.85000000e+02]),
 'a5_plus': array([ 100., -540.,  585.]),
 'a6_minus': array([ 3.24531402e-14, -5.30000000e+02,  5.85000000e+02]),
 'a6_origin': array([ 3.85763742e-14, -6.30000000e+02,  5.85000000e+02])}

In [7]:

def norm(x):
    return x/max(np.linalg.norm(x),10**-9)

def create_graph(scale_shift,joints_ref,position = None, direction= None, anchor = False):

    axes, translation, rotation, step_vect, e0, e1, e2, e3 = load_kuka_KR_10_R1100(scale_shift,joints_ref)
    
    G = nx.Graph()

    G.add_node(1,pos_ref=e1)
    G.add_node(2,pos_ref=e2)
    G.add_edge(1,2, weight  = np.linalg.norm(e1-e2), dir_ref  = -norm(e1-e2))


    count = 2

    for i, ax in enumerate(axes):
        v1,v2 = step_vect[ax]

        transform = []

        for j in range(i+1):
            transform.append(translation[j])
            transform.append(rotation[j])
        transform = np.array(transform)
        transformation = np.linalg.multi_dot(transform)

        G.add_node(count+1,pos_ref=(transformation@T(v2))[:3,3])
        G.add_node(count+2,pos_ref=(transformation@T(v1))[:3,3])

        node_pos_ref = nx.get_node_attributes(G,'pos_ref')

        G.add_edge(count+2,count+1, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count+2]))
        
        G.add_edge(count,count+1, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count]))
        G.add_edge(count-1,count+1, weight = np.linalg.norm(node_pos_ref[count+1]-node_pos_ref[count-1]))

        G.add_edge(count,count+2, weight = np.linalg.norm(node_pos_ref[count+2]-node_pos_ref[count]))
        G.add_edge(count-1,count+2, weight = np.linalg.norm(node_pos_ref[count+2]-node_pos_ref[count-1]))

        count = count + 2

    mapping = {1: "x", 2: "y", 3: "a1_plus", 4: "a1_origin", 5: "a2_minus", 6: "a2_plus", 7: "a3_minus", 8:"a3_plus", 
            9: "a4_minus", 10:"a4_origin", 11: "a5_minus", 12:"a5_plus", 13: "a6_minus", 14:"a6_origin"}

    G = nx.relabel_nodes(G, mapping)

    G.add_edge('a1_origin','a6_origin', weight = np.linalg.norm(position))

    G.add_edge('x','a6_origin', weight = np.linalg.norm(position-e1))
    G.add_edge('y','a6_origin', weight = np.linalg.norm(position-e2))
    G.add_edge('a1_plus','a6_origin', weight = np.linalg.norm(position-e3))
               
    if direction is not None:
        G.add_edge('a1_origin','a6_minus', weight = np.linalg.norm(position-scale_shift*direction))

        G.add_edge('x','a6_minus', weight = np.linalg.norm(position-scale_shift*direction-e1))
        G.add_edge('y','a6_minus', weight = np.linalg.norm(position-scale_shift*direction-e2))
        G.add_edge('a1_plus','a6_minus', weight = np.linalg.norm(position-scale_shift*direction-e3))
    else:
        direction = None

    if anchor:
        if direction is not None:
            fix_edges = [('x', 'y'), ('x', 'a1_plus'), ('x', 'a1_origin'), ('x', 'a6_origin'), ('x', 'a6_minus'),
                        ('y', 'a1_plus'), ('y', 'a1_origin'), ('y', 'a6_origin'), ('y', 'a6_minus'),
                        ('a1_plus', 'a1_origin'),('a1_plus', 'a6_origin'), ('a1_plus', 'a6_minus'),
                        ('a6_minus', 'a6_origin'),('a1_origin', 'a6_origin'),('a1_origin', 'a6_minus')]
        else:
            fix_edges = [('x', 'y'), ('x', 'a1_plus'), ('x', 'a1_origin'), ('x', 'a6_origin'), 
                        ('y', 'a1_plus'), ('y', 'a1_origin'), ('y', 'a6_origin'), 
                        ('a1_plus', 'a1_origin'),('a1_plus', 'a6_origin'),
                        ('a1_origin', 'a6_origin')]

        other_edges = []
        for e in G.edges():
            if e not in fix_edges:
                other_edges.append(e)

        if direction is not None:
            direction_ref = {('x', 'y'): norm(e2-e1) , ('x', 'a1_plus'): norm(e3-e1), ('x', 'a1_origin'): norm(-e1), 
                            ('x', 'a6_origin'): norm(position-e1), ('x', 'a6_minus'): norm(position - np.linalg.norm(e1)*direction -e1),
                        ('y', 'a1_plus'): norm(e3-e2), ('y', 'a1_origin'): norm(-e2), ('y', 'a6_origin'): norm(position-e2),
                        ('y', 'a6_minus'): norm(position - np.linalg.norm(e1)*direction - e2),
                        ('a1_plus', 'a1_origin'): norm(-e3),('a1_plus', 'a6_origin'): norm(position-e3),
                        ('a1_plus', 'a6_minus'): norm(position - np.linalg.norm(e1)*direction-e3),
                        ('a6_minus', 'a6_origin'): norm(direction),('a1_origin', 'a6_origin'):norm(position),
                        ('a1_origin', 'a6_minus'): norm(position - np.linalg.norm(e1)*direction)}
        else:
            direction_ref = {('x', 'y'): norm(e2-e1) , ('x', 'a1_plus'): norm(e3-e1), ('x', 'a1_origin'): norm(-e1), 
                            ('x', 'a6_origin'): norm(position-e1), 
                        ('y', 'a1_plus'): norm(e3-e2), ('y', 'a1_origin'): norm(-e2), ('y', 'a6_origin'): norm(position-e2),
                        ('a1_plus', 'a1_origin'): norm(-e3),('a1_plus', 'a6_origin'): norm(position-e3),('a1_origin', 'a6_origin'):norm(position)}
        
        pos_ref = nx.get_node_attributes(G,'pos_ref')

        new_edge_order = fix_edges.copy()
        new_edge_order.extend(other_edges.copy())

        A_ = []
        for e in fix_edges:
            A_.append(direction_ref[e])
        A_ = np.array(A_).transpose()
    else:
        fix_edges = []
        other_edges = [e for e in G.edges()]
        new_edge_order = other_edges.copy()
        A_ = None
        
        n_anchor = 0
        n = len(new_edge_order)

    weight = nx.get_edge_attributes(G, "weight")
    D = np.diag([weight[i] for i in new_edge_order])

    C = incidence_matrix_(G,oriented=True, edgelist=new_edge_order).toarray()
    C = np.array(C)

    n = len(other_edges)
    n_anchor = len(fix_edges)
    
    Y_init = np.zeros((3,n))

    for i, e in enumerate(other_edges):
        i = i 
        p1 = G.nodes[e[0]]['pos_ref']
        p2 = G.nodes[e[1]]['pos_ref']
        Y_init[:,i] = (p2-p1)/max(np.linalg.norm(p2-p1),10**-9)
        
        return G, fix_edges, other_edges, new_edge_order, n,n_anchor, A_, D,C,Y_init, e0, e1, e2, e3

In [8]:

def simple_IK(d,C,D,A_,n, n_anchor,max_iter,ind,W=None,Y_init=None,use_rand=False):
    if  W is None:
        W = np.eye(D.shape[0])
    if n_anchor > 0:    
        if d > 3:
            A = np.zeros((d,n_anchor))
            A[:3,:] = A_[:3,:]
        else:
            if A_.shape[0] == d:
                A = A_
            elif A_.shape[0] > d:
                A = A_[:d,:]
            else:
                A = np.zeros((d,n_anchor))
                A[:3,:] = A_[:3,:]

        manifold_ = manifolds.Product([manifolds.ConstantFactory(A) , manifolds.Oblique(d,n)])
    else:
        manifold_ = manifolds.Oblique(d,n)

    Q2 = -D@(W@C.transpose()@np.linalg.pinv(C@W@C.transpose()))@C@W@D
    Q1 = D@W@D

    if n_anchor > 0:
        zero = np.zeros_like(A)
        @pymanopt.function.numpy(manifold_)
        def cost(A,Y_):
            print(A.shape,Y_.shape)
            Y = np.concatenate([A,Y_],axis=1)
            return np.trace(Q1 + Q2@Y.transpose()@Y) 

        @pymanopt.function.numpy(manifold_)
        def euclidean_gradient(A,Y_):
            Y = np.concatenate([A,Y_],axis=1)
            grad = 2*Q2[n_anchor:,:]@Y.transpose()
            return [zero,grad.transpose() ]
                                
        @pymanopt.function.numpy(manifold_)
        def euclidean_hessian(A,Y_,U_A, U):
            U_ = np.concatenate([U_A,U],axis=1)
            hess = 2*Q2[n_anchor:,:]@U_.transpose()
            return [zero,hess.transpose()]
    else:
        @pymanopt.function.numpy(manifold_)
        def cost(Y):
            return np.trace(Q1 + Q2@Y.transpose()@Y) 

        @pymanopt.function.numpy(manifold_)
        def euclidean_gradient(Y):
            grad = 2*Q2@Y.transpose()
            return grad.transpose() 
                                
        @pymanopt.function.numpy(manifold_)
        def euclidean_hessian(Y,U):
            hess = 2*Q2@U.transpose()
            return hess.transpose()


    problem = Problem(manifold=manifold_, 
                      cost=cost,
                      euclidean_gradient=euclidean_gradient, 
                      euclidean_hessian=euclidean_hessian)

    optimizer = optimizers.TrustRegions(max_iterations=max_iter,use_rand=use_rand)
    
    if  Y_init is None:
        Y_init_ = manifold_.random_point()
    elif len(Y_init) == 2:
        if Y_init[0].shape[0] == d:
            print(1)
            Y_init_ = Y_init
        elif Y_init[0].shape[0] < d:
            print(2)
            Y_init_ = np.zeros((d,n))
            Y_init_[:Y_init[0].shape[0],:] = Y_init[1]
            Y_init_ = [A, Y_init_]
        else:
            print(3)
            U,S,V = np.linalg.svd(Y_init[1],full_matrices=False)
            Y_init_ = [A, U[:d,:d]@np.diag(S)[:d,:]@V/np.maximum(np.linalg.norm(U[:d,:d]@np.diag(S)[:d,:]@V,axis=0),10**-9)]
    else:
        if Y_init.shape[0] == d:
            print(1)
            Y_init_ = Y_init
        elif Y_init.shape[0] < d:
            print(2)
            Y_init_ = np.zeros((d,n))
            Y_init_[:Y_init.shape[0],:] = Y_init
        else:
            print(3)
            U,S,V = np.linalg.svd(Y_init,full_matrices=False)
            Y_init_ = U[:d,:d]@np.diag(S)[:d,:]@V/np.maximum(np.linalg.norm(U[:d,:d]@np.diag(S)[:d,:]@V,axis=0),10**-9)
    
    Y_star = optimizer.run(problem,initial_point=Y_init_).point

    return Y_star


In [13]:

joints_ref=np.array([90.000,-90,90,0,0,0])*np.pi/180
scale_shift = 8*10**2

position = pose_robot_(joints_ref,'camera')[:3,3] # np.array([200,200,695])#
print(position)
direction = pose_robot_(joints_ref,'camera')[:3,0] #np.array([1,0,0])
direction = direction/np.linalg.norm(direction)

G, fix_edges, other_edges, new_edge_order, n,n_anchor, A_, D,C,Y_init, e0, e1, e2, e3 = create_graph(scale_shift,joints_ref,position, direction= direction,anchor =False)

[ 3.85763742e-14 -6.30000000e+02  5.85000000e+02]


In [16]:
W = np.diag(np.concatenate([np.ones(n_anchor),np.ones(n)]))
#Y_D10 = simple_IK(3,C,D,A_,n, n_anchor,max_iter=1000,W=W,Y_init=None)
#Y_D5 = simple_IK(5,C,D,A_,n, n_anchor=0,max_iter=1000,ind=[14,19],W=W,Y_init=None) #[A_,Y_init])
Y_D4 = simple_IK(4,C,D,A_,n, n_anchor,max_iter=2000,ind=[14,19],W=W,Y_init=None)
Y_D3 = simple_IK(3,C,D,A_,n, n_anchor,max_iter=3000,ind=[14,19],W=W,Y_init=Y_D4,use_rand = False)


Optimizing...
                                            f: +2.015643e+07   |grad|: 3.739207e+06
acc TR+   k:     1     num_inner:     0     f: +1.155004e+07   |grad|: 3.701044e+06   negative curvature
acc       k:     2     num_inner:     0     f: +3.853218e+06   |grad|: 2.551907e+06   exceeded trust region
acc       k:     3     num_inner:     2     f: +1.715619e+06   |grad|: 1.717847e+06   exceeded trust region
acc       k:     4     num_inner:     3     f: +9.023834e+05   |grad|: 1.298547e+06   exceeded trust region
REJ TR-   k:     5     num_inner:     6     f: +9.023834e+05   |grad|: 1.298547e+06   negative curvature
acc TR+   k:     6     num_inner:     1     f: +1.758852e+05   |grad|: 3.785225e+05   exceeded trust region
acc       k:     7     num_inner:     6     f: +1.253797e+05   |grad|: 3.879029e+05   exceeded trust region
acc       k:     8     num_inner:    14     f: +4.689926e+04   |grad|: 2.709194e+05   exceeded trust region
acc       k:     9     num_inner:     9     

In [20]:
Y_star = Y_D3
#X = direction_to_position(np.concatenate([Y_star[0][:3,:],U[:3,:3]@np.diag(S)[:3,:]@V],axis=1),C,D)
if len(Y_star) == 2:
    X = direction_to_position(np.concatenate([Y_star[0],Y_star[1]],axis=1),C,D)
else:
    X = direction_to_position(Y_star,C,D)


R = rotation_matrix_from_vectors(X[1,:], norm(e2))
R_ = rotation_matrix_from_vectors(R@X[0,:], norm(e1))
Rz_flip = np.diag((1,1,1))

print(X@R.transpose()@Rz_flip)
print(np.linalg.norm(X[-1,:]@R.transpose()@R_.transpose()@Rz_flip-position))
print(np.linalg.norm((X[-1,:]-X[-2,:])@R.transpose()@R_.transpose()@Rz_flip/scale_shift-direction))
X = X@R.transpose()@Rz_flip

(3, 39) (14, 39) (39, 39)
[[ 7.99999999e+02  1.01663490e-06  1.24685742e-06]
 [ 7.17970894e-23  8.00000005e+02 -4.85734308e-14]
 [ 8.10733959e-06  2.22551342e-05  8.00000012e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 2.39571431e-02 -2.49999895e+01  1.09627762e-05]
 [-7.99975746e+02 -2.57644052e+01  1.93830510e-05]
 [ 5.54548834e-01 -5.80500983e+02  7.08407920e+01]
 [-7.99444657e+02 -5.81366148e+02  7.00462284e+01]
 [ 8.20425311e-01 -5.64096866e+02 -2.14782794e+02]
 [ 1.60120080e-04 -5.40000047e+02  5.84853792e+02]
 [ 1.58838689e-04 -5.40000048e+02  5.84853793e+02]
 [-7.99999419e+02 -5.40000020e+02  5.84033155e+02]
 [-1.07208131e-05  1.69999988e+02  5.84999990e+02]
 [ 9.35100259e-07 -6.29999977e+02  5.85000025e+02]]
3.420752409242295e-05
6.371575038278272e-08


In [18]:
def direction_to_position(Y,C,D):
    print(Y.shape, C.shape, D.shape)
    X = np.linalg.pinv(C@C.transpose())@C@D@Y.transpose()
    X = X - X[3,:]
    
    #if X[0,0]<0:
      #  X = -X
    
    v = X[0,:]
    R = rotation_matrix_from_vectors(v/np.linalg.norm(v),e1/np.linalg.norm(e1))
    
    return X@R.transpose()

In [ ]:
print(np.arccos(Y_D3[1].transpose()[19-n_anchor,:]@norm(e2))*180/np.pi)
print(Y_D3[1].transpose()[19-n_anchor,:], norm(e2))
print(Rz(np.arccos(Y_star[1].transpose()[19-n_anchor,:]@norm(e2)))[:3,:3]@norm(e2)-Y_star[1].transpose()[19-n_anchor,:])

0.912029275610479
[1.59172395e-02 9.99873313e-01 7.95120020e-06] [0. 1. 0.]
[1.31273291e-12 0.00000000e+00 1.00159986e-07]


In [19]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

In [ ]:

joint = sol_to_config_kuka_KR_10_R1100(G,Y_D3,new_edge_order,n_anchor,scale_shift)
joint.append(0)
#joint[1] = -joint[1]
#joint[-2] = np.pi-joint[-2]
print([180/np.pi*j for j in joint])
print(pose_robot_(joint,'camera'))
print(position,direction)
print(np.linalg.norm(pose_robot_(joint,'camera')[:3,3] - position), np.linalg.norm(direction - pose_robot_(joint,'camera')[:3,0]))